In [1]:
#Se importan las librerías necesarias
import requests
import pandas as pd
from sqlalchemy import create_engine, String, Integer, Float, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, Mapped, mapped_column, relationship

# **Extraccion de los datos**
#### Inicialmente se extraen los datos utilizando una api que proporciona la pagina de la que se extraen los datos

In [2]:
def obtener_datos(url: str) -> pd.DataFrame:
    '''
    Obtiene los datos a partir de una petición a la API de la página donde se encuentran los datos

    Args:
        url (str): Dirección url de la página que contiene los datos

    Returns:
        df (pd.DataFrame): DataFrame con los datos
    '''
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

# **Limpieza de datos**
#### Las siguientes funciones permiten limpiar el DataFrame

In [3]:
def renombrar_columnas(df: pd.DataFrame, columnas: dict) -> pd.DataFrame:
    '''
    Renombra una columna especifica de un DataFrame

    Args: 
        df (pd.DataFrame): DataFrame de interés
        columnas (dict): Diccionario con la siguiente estructura {"columna_a_renombrar": "nuevo_nombre"}
    
    Returns: 
        df (pd.DataFrame): DataFrame con las columnas renombradas
    '''
    df = df.rename(columns=columnas)
    return df

def eliminar_columnas(df: pd.DataFrame, columnas: list) -> pd.DataFrame:
    '''
    Elimina una o varias columnas de un DataFrame

    Args:
        df (pd.DataFrame): DataFrame de interés
        columnas (dict): Lista con las columnas que se desean eliminar

    Returns:
        df (pd.DataFrame): DataFrame sin las columnas seleccionadas
    '''
    df = df.drop(columns=columnas)
    return df

def correccion_ciudad(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Corrige el formato en el que viene la columna "ciudad_domicilio" para eliminar la redundancia con los registros de otras columnas

    Args:
        df (pd.DataFrame): DataFrame de interés

    Returns:
        df (pd.DataFrame): DataFrame con la columna "ciudad_domicilio" en un formato correcto
    '''
    ciudad = df["ciudad_domicilio"].str.split("-")
    ciudad = ciudad.apply(lambda x: x[0])
    df["ciudad_domicilio"] = ciudad
    return df

def reemplazar_valores(df: pd.DataFrame, columna: str, reemplazo: dict) -> pd.DataFrame:
    '''
    Reemplaza valores en una columna de interés

    Args:
        df (pd.DataFrame): DataFrame de interés
        columna (str): Columna en la que se quiere hacer los reemplazos
        reemplazo (dict): Diccionario con el siguiente estructura {"valor_a_reemplazar": "valor_de_reemplazo"}
    
    Returns:
        df (pd.DataFrame): DataFrame con valores reemplazados en la columna de interés
    
    '''
    df[columna] = df[columna].replace(reemplazo)
    return df

def cambiar_tipos_valores(df: pd.DataFrame, columnas: list, tipo: type) -> pd.DataFrame:
    '''
    Cambia el tipo de valor en una columna especifica

    Args:
        df (pd.DataFrame): DataFrame de interés
        columnas (list): Lista de columnas en las cuales se va a cambiar el tipo de dato
        tipo (type): Tipo de dato en el que se desean los datos

    Returns:
        df (pd.DataFrame): DataFrame con los valores de la columna en el tipo correcto de dato
    '''
    df[columnas] = df[columnas].astype(tipo)
    return df

def cambiar_valores_numericos(df: pd.DataFrame, columnas: list) -> pd.DataFrame:
    '''
    Cambia el formato en el que se presentan los valores de una columna especifica

    Args:
        df (pd.DataFrame): DataFrame de interés
        columnas (list): Lista de columnas en las cuales se va a cambiar el formato de los datos

    Returns:
        df (pd.DataFrame): DataFrame con los valores de la columna en el formato correcto
    '''
    df[columnas] = df[columnas].apply(lambda x: x.str.replace("$",""))
    return df

# **Transformaciones**
#### Las siguientes funciones permiten realizar transformaciones sobre las columnas del DataFrame para obtener datos válidos

In [4]:
def df_columna(df: pd.DataFrame, columna: str) -> pd.DataFrame:
    '''
    Extrae los valores unicos de una columna de interés del DataFrame

    Args:
        df (pd.DataFrame): DataFrame de interés
        columna (str): Columna de interés
    
    Returns: 
        df (pd.Dataframe): DataFrame que contiene solo los valores unicos de "columna"
    '''
    nuevo_df = df[columna].unique()
    df_col = pd.DataFrame(nuevo_df, columns=[columna])
    return df_col

def indices_de_tabla(df: pd.DataFrame, df_referencia: pd.DataFrame, columna_origen: str, columna_destino: str) -> pd.DataFrame:
    '''
    Cambia los valores de columna_destino en df por los respectivos id asociados a los valores de columna_origen en df_referencia

    Args:
       df (pd.DataFrame): DataFrame de interés
       df_referencia (pd.DataFrame): DataFrame con los valores unicos y sus respectivos id
       columna_origen (str): Columna que contiene los id a cambiar en df_referencia
       columna_destino (str): Columna que contiene los valores a cambiar en df  
    
    Returns:
        df_indices (pd.DataFrame): DataFrame donde los valores de columna_destino corresponden a sus respectivos id
    '''
    indices = df_referencia.to_dict()[columna_origen]
    indices = {valor: clave for clave, valor in indices.items()}
    df[columna_destino] = df[columna_destino].map(indices)
    df_indices = df.drop_duplicates()
    return df_indices

def crear_clave_compuesta(df: pd.DataFrame, columna: str, nombre_id: str) -> pd.DataFrame:
    '''
    Crea una columna que contiene una clave compuesta en el caso de ser necesario

    Args:
        df (pd.DataFrame): DataFrame de interés
        columna (str): Columna cuyos valores se desean cambiar por las claves compuestas
        nombre_id (str): Nombre que va a llevar la nueva columna con las claves compuestas
    
    Returns:
        df (pd.DataFrame): DataFrame con los valores de la columna de interés cambiados por su respectiva clave compuesta
    '''
    df["id_" + nombre_id] = df.index.astype(str) + df[columna].astype(str)
    df = df.set_index("id_" + nombre_id)
    df = df.drop(columns=[columna])
    return df

def ubicacion(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Obtiene las combinaciones unicas de las columnas departamento y ubicacion del DataFrame df

    Args:
        df (pd.DataFrame): DataFrame de interés
    
    Returns:
        df (pd.DataFrame): DataFrame con ubicaciones unicas
    '''
    ubicacion = df[["departamento", "ubicacion"]].drop_duplicates()
    return ubicacion

def creacion_tabla(df: pd.DataFrame, nombre: str, conexion: str):
    '''
    Crea una tabla sql a partir de un DataFrame df

    Args:
        df (pd.DataFrame): DataFrame de interés
        nombre (str): Nombre de la tabla
        conexion (str): Nombre de la conexion a la base de datos

    Returns:
        Tabla SQL en SQLite
    '''
    df.to_sql(nombre, con=conexion, if_exists="append", index=True)
    
def creacion_diccionario(df: pd.DataFrame) -> dict:
    '''
    Crea un diccionario a partir de un DataFrame {indice: registro}
    
    Args:
        df (pd.DataFrame): DataFrame de interés
    
    Returns:
        diccionario (dict): Diccionario con información del DataFrame df
    '''
    diccionario = df.to_dict(orient="records")
    return diccionario

### La función main() establece el flujo de trabajo para el proceso de limpieza y transformación 

In [5]:
def main(url: str) -> pd.DataFrame:
    data = obtener_datos(url)
    data = correccion_ciudad(data)
    data = reemplazar_valores(data, "departamento_domicilio", {"BOGOTA D.C.": "CUNDINAMARCA"})
    data = reemplazar_valores(data, "ciudad_domicilio", {"BOGOTÁ": "BOGOTA D.C."})
    data = cambiar_valores_numericos(data, ['ingresos_operacionales', 'ganancia_p_rdida', 'total_activos', 'total_pasivos', 'total_patrimonio'])
    data = cambiar_tipos_valores(data, ["ciiu"], int)
    data = cambiar_tipos_valores(data, ['ingresos_operacionales', 'ganancia_p_rdida', 'total_activos', 'total_pasivos', 'total_patrimonio'], float)
    data = eliminar_columnas(data, ["regi_n"])
    data = renombrar_columnas(data, {"raz_n_social": "razon_social", "departamento_domicilio": "departamento", "ciudad_domicilio": "ubicacion", "ganancia_p_rdida": "ganancia_perdida", "a_o_de_corte": "año_corte"})
    return data

## Se crea la conexión a la base de datos SQLite

In [6]:
engine = create_engine('sqlite:///C:\\Users\\maico\\Documents\\Codigos varios\\Python\\Cosas\\base_datos.db')
Session = sessionmaker(bind=engine)
Base = declarative_base()

#### Se crean las clases que representan las tablas y se establece el modelo de datos

In [7]:
class Razon_social(Base):
    __tablename__ = "razon_social"

    id: Mapped[int] = mapped_column(primary_key=True, unique=True)
    razon_social: Mapped[str] = mapped_column(String(225), nullable=False)
    empresa_razon: Mapped["Empresas"] = relationship(uselist=False, back_populates="id_razonsocial") #Relación 1 a 1

class Supervisores(Base):
    __tablename__ = "supervisores"

    id: Mapped[int] = mapped_column(primary_key=True, unique=True)
    supervisor: Mapped[str] = mapped_column(String(225),nullable=False)
    empresa_supervisor: Mapped[list["Empresas"]] = relationship(back_populates="id_super") #Relación 1 a n

class Macrosectores(Base):
    __tablename__ = "macrosectores"

    id: Mapped[int] = mapped_column(primary_key=True, unique=True)
    macrosector: Mapped[str] = mapped_column(String(225),nullable=False)
    empresa_macrosector: Mapped[list["Empresas"]] = relationship(back_populates="id_macro")

class Departamentos(Base):
    __tablename__ = "departamentos"

    id: Mapped[int] = mapped_column(primary_key=True, unique=True)
    departamento: Mapped[str] = mapped_column(String(225),nullable=False)
    ubi_depto: Mapped[list["Ubicaciones"]] = relationship(back_populates="depto")

class Ubicaciones(Base):
    __tablename__ = "ubicaciones"

    id_ubicacion: Mapped[str] = mapped_column(String(225), nullable=False, primary_key=True, unique=True)
    departamento: Mapped[int] = mapped_column(Integer, ForeignKey("departamentos.id"))
    ubicacion: Mapped[str] = mapped_column(String(225),nullable=False)
    
    empresa_ubicacion: Mapped[list["Empresas"]] = relationship(back_populates="id_ubi")
    depto: Mapped["Departamentos"] = relationship(back_populates="ubi_depto")

class Empresas(Base):
    __tablename__ = "empresas"

    id: Mapped[int] = mapped_column(primary_key=True, unique=True)
    nit: Mapped[int] = mapped_column(Integer, nullable=False)
    id_razon_social: Mapped[int] = mapped_column(Integer, ForeignKey("razon_social.id"))
    id_supervisor: Mapped[int] = mapped_column(Integer, ForeignKey("supervisores.id"))
    id_ubicacion: Mapped[str] = mapped_column(String, ForeignKey("ubicaciones.id_ubicacion"))
    ciiu: Mapped[int] = mapped_column(Integer, nullable=False)
    id_macrosector: Mapped[int] = mapped_column(Integer, ForeignKey("macrosectores.id"))
    ingresos_operacionales: Mapped[float] = mapped_column(Float, nullable=False)
    ganancia_perdida: Mapped[float] = mapped_column(Float, nullable=False)
    total_activos: Mapped[float] = mapped_column(Float, nullable=False)
    total_pasivos: Mapped[float] = mapped_column(Float, nullable=False)
    total_patrimonio: Mapped[float] = mapped_column(Float, nullable=False)
    año_corte: Mapped[str] = mapped_column(String, nullable=False)
    id_razonsocial: Mapped["Razon_social"] = relationship(uselist=False, back_populates="empresa_razon")
    id_ubi: Mapped["Ubicaciones"] = relationship(back_populates="empresa_ubicacion")
    id_super: Mapped["Supervisores"] = relationship(back_populates="empresa_supervisor")    
    id_macro: Mapped["Macrosectores"] = relationship(back_populates="empresa_macrosector")
    

#### Se realiza la limpieza y transformación de los datos

In [8]:
url = "https://www.datos.gov.co/resource/6cat-2gcs.json?$limit=30000"
data = main(url)

#### Se crean los DataFrames que servirán para crear las tablas

In [9]:
df_razon_social = df_columna(data, "razon_social")
df_supervisor = df_columna(data, "supervisor")
df_departamento = df_columna(data, "departamento")
df_macrosector = df_columna(data, "macrosector")

#Creación del df_ubicacion
df_ubicacion = ubicacion(data)
df_ubicacion = indices_de_tabla(df_ubicacion, df_departamento, "departamento", "departamento")
ubicacion_id = crear_clave_compuesta(df_ubicacion, "departamento", "ubicacion")

#Cambio de los valores por sus respectivas claves
data = indices_de_tabla(data, df_razon_social, "razon_social", "razon_social")
data = indices_de_tabla(data, df_supervisor, "supervisor", "supervisor")
data = indices_de_tabla(data, df_macrosector, "macrosector", "macrosector")
data = indices_de_tabla(data, ubicacion_id, "ubicacion", "ubicacion")

data = eliminar_columnas(data, ["departamento"])
data = renombrar_columnas(data, {"razon_social":"id_razon_social", "supervisor": "id_supervisor", "ubicacion": "id_ubicacion", "macrosector": "id_macrosector"})

#### Se crean los diccionario a partir de los cuales se cargarán los datos a las tablas

In [10]:
#Razon social
df_razon_social = df_razon_social.reset_index().rename(columns={"index": "id"})
dict_razon_social = creacion_diccionario(df_razon_social)

#Supervisor
df_supervisor = df_supervisor.reset_index().rename(columns={"index": "id"})
dict_supervisor = creacion_diccionario(df_supervisor)

#Departamentos
df_departamento = df_departamento.reset_index().rename(columns={"index": "id"})
dict_departamento = creacion_diccionario(df_departamento)

#Macrosector
df_macrosector = df_macrosector.reset_index().rename(columns={"index": "id"})
dict_macrosector = creacion_diccionario(df_macrosector)

#Ubicación
df_ubicacion = df_ubicacion.reset_index().rename(columns={"index": "id"})
dict_ubicacion = creacion_diccionario(df_ubicacion)

#Empresas
df_empresas = data.reset_index().rename(columns={"index": "id"})
dict_empresas = creacion_diccionario(df_empresas)

#### Se crean las tablas en la base de datos y se cargan los datos de cada uno de los diccionarios creados anteriormente 

In [11]:
Base.metadata.create_all(engine)

with Session() as session:
    session.bulk_insert_mappings(Razon_social, dict_razon_social)
    session.bulk_insert_mappings(Supervisores, dict_supervisor)
    session.bulk_insert_mappings(Departamentos, dict_departamento)
    session.bulk_insert_mappings(Macrosectores, dict_macrosector)
    session.bulk_insert_mappings(Ubicaciones, dict_ubicacion)
    session.bulk_insert_mappings(Empresas, dict_empresas)
    session.commit()